This note book is created for the coursera capstone project for Applied Data Science course with IBM and will be mainly use for that.

In [69]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
import requests 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import folium # map rendering library
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


## 1) See if there is any Persian restuarants in London

In [18]:
CLIENT_ID = 'APPATQVPK2DBJU4YZ2G5TCSUKKZ31EYEFH1OT5TWQJC1EQ4P' # your Foursquare ID
CLIENT_SECRET = 'YYUXMBIKMLM3WDD4ZJOVYEVSYE5EJMHHXA3ACCIDZKTWMTRX' # your Foursquare Secret
VERSION = '20191129' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: APPATQVPK2DBJU4YZ2G5TCSUKKZ31EYEFH1OT5TWQJC1EQ4P


In [22]:
LIMIT = 500 # Maximum is 100
cities = ["London, GB"]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "52e81612bcbc57f1066b79f7") # Persian Food CATEGORY ID
    results[city] = requests.get(url).json()

In [23]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [24]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Persian food places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Persian food places in London, GB =  64
Showing Top 100


In [25]:
maps[cities[0]]

### As we can see, there are 64 Persian restuarant in London distributed like this map. It may make it risky to invest in such bussiness and thus we need to analyze more.

## 2) Check Ethnicity By Borough

In [36]:
# Read ethnicity data

In [58]:
df_popu = pd.read_excel('ethnic-groups-by-borough.xls')
df_popu.shape

(47, 7)

In [59]:
df_popu.head()

,Code,Area,White,Asian,Black,Mixed/ Other,Total
0,E09000002,Barking and Dagenham,109000.0,54000.0,36000,15000.0,215000.0
1,E09000003,Barnet,250000.0,57000.0,30000,54000.0,390000.0
2,E09000004,Bexley,195000.0,17000.0,21000,15000.0,248000.0
3,E09000005,Brent,102000.0,107000.0,62000,56000.0,328000.0
4,E09000006,Bromley,267000.0,15000.0,21000,28000.0,330000.0


In [60]:
df_popu = df_popu.dropna(how='any')

In [61]:
df_popu.head()

,Code,Area,White,Asian,Black,Mixed/ Other,Total
0,E09000002,Barking and Dagenham,109000.0,54000.0,36000,15000.0,215000.0
1,E09000003,Barnet,250000.0,57000.0,30000,54000.0,390000.0
2,E09000004,Bexley,195000.0,17000.0,21000,15000.0,248000.0
3,E09000005,Brent,102000.0,107000.0,62000,56000.0,328000.0
4,E09000006,Bromley,267000.0,15000.0,21000,28000.0,330000.0


In [62]:
df_popu = df_popu.set_index(['Code'])
df_popu.head()

,Area,White,Asian,Black,Mixed/ Other,Total
Code,,,,,,
E09000002,Barking and Dagenham,109000.0,54000.0,36000,15000.0,215000.0
E09000003,Barnet,250000.0,57000.0,30000,54000.0,390000.0
E09000004,Bexley,195000.0,17000.0,21000,15000.0,248000.0
E09000005,Brent,102000.0,107000.0,62000,56000.0,328000.0
E09000006,Bromley,267000.0,15000.0,21000,28000.0,330000.0


In [63]:
df_popu['Population'] = df_popu.mean(numeric_only=True, axis=1)

In [64]:
df_popu.head()

,Area,White,Asian,Black,Mixed/ Other,Total,Population
Code,,,,,,,
E09000002,Barking and Dagenham,109000.0,54000.0,36000,15000.0,215000.0,98250.0
E09000003,Barnet,250000.0,57000.0,30000,54000.0,390000.0,187750.0
E09000004,Bexley,195000.0,17000.0,21000,15000.0,248000.0,118750.0
E09000005,Brent,102000.0,107000.0,62000,56000.0,328000.0,148250.0
E09000006,Bromley,267000.0,15000.0,21000,28000.0,330000.0,160000.0


In [65]:
df_popu.loc[df_popu['Population'].idxmax()]

Area            United Kingdom
White                 5.51e+07
Asian                4.655e+06
Black                  2133000
Mixed/ Other         2.273e+06
Total               6.4162e+07
Population         3.15475e+07
Name: K02000001, dtype: object

### As the Newham borough is having highest asian population, we will consider this borough as our preferred location for our asian restuarant.

In [66]:
df_Newham_asians = df_popu.loc[df_popu['Area'] == 'Newham']

In [67]:
df_Newham_asians

,Area,White,Asian,Black,Mixed/ Other,Total,Population
Code,,,,,,,
E09000025,Newham,111000.0,166000.0,47000,27000.0,351000.0,163750.0


## Preferred location for Persian restuarant - Newham Borough
#### As the borough Newham is having high asian population, we will consider to Segment and Cluster only the neighbourhoods of this borough. For that we have to get the latitude and longitude details of all the areas(neighbourhoods) of Newham borough.
#### Read the latitude and longitude coordinates of all the neighborhoods(areas) in Newham Borough¶

In [70]:
from urllib.request import urlopen
import re
URL = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

codes = []
areas_list = []
href_links_list = []
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Newham':
        codes.append(code)
        areas_list.append((borough_name,area_name,code))

                
for link in soup.findAll('a', attrs={'href': re.compile("^https://tools.wmflabs.org")}):
            htext = link.text
            if htext in codes:
                hlink = link.get('href')
                href_links_list.append((htext, hlink))

In [71]:
# install and import folium library 
!pip -q install folium
import folium

1) Download and Explore the Dataset

In [72]:
URL = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

df_list = []
# print(soup)
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name.split('[')
        borough_name = borough_name[0]
        borough_name = borough_name.strip()
        
        ll = data8.get_text()
        ll = ll.split('/')
        lat_long = ll[2]
        lat_long = lat_long.split('(')
        lat_long = lat_long[0]
        lat_long = lat_long.split(';')
        latitude = lat_long[0]
        latitude = latitude.strip()
        longitude = lat_long[1]
        longitude = longitude.strip()
        longitude = longitude.replace(u'\ufeff', '')
        latitude = float(latitude)
        longitude = float(longitude)

#       Append the borough name, latitude and logitude in a list
        df_list.append((borough_name, latitude, longitude))
    except IndexError:pass

In [73]:
df_boroughs = pd.DataFrame(df_list, columns=['Borough', 'Latitude' , 'Longitude'])

In [74]:
df_boroughs.shape

(32, 3)

In [75]:
df_boroughs.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


In [76]:
df_boroughs.dtypes

Borough       object
Latitude     float64
Longitude    float64
dtype: object

In [77]:
df_boroughs.loc[df_boroughs['Borough'] == 'Newham']


,Borough,Latitude,Longitude
23,Newham,51.5077,0.0469


In [78]:
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

C:\Users\AID-FLOATER-2015-02\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of London City are 51.5073219, -0.1276474.


## Create a map of London with Boroughs superimposed on top

In [81]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

In [82]:
# add markers to map
for lat, lng, borough in zip(df_boroughs['Latitude'], df_boroughs['Longitude'], df_boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_london)  

map_london

## Preferred location for Persian restuarant - Newham Borough

In [83]:
from urllib.request import urlopen
import re
URL = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

codes = []
areas_list = []
href_links_list = []
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Newham':
        codes.append(code)
        areas_list.append((borough_name,area_name,code))

                
for link in soup.findAll('a', attrs={'href': re.compile("^https://tools.wmflabs.org")}):
            htext = link.text
            if htext in codes:
                hlink = link.get('href')
                href_links_list.append((htext, hlink))

#### Create a DataFrame from the Areas list

In [86]:
df_areas = pd.DataFrame(areas_list, columns=['Borough', 'Area', 'Code'])

In [87]:
df_areas

,Borough,Area,Code
0,Newham,Beckton,TQ435815
1,Newham,Canning Town,TQ405815
2,Newham,Custom House,TQ408807
3,Newham,East Ham,TQ425835
4,Newham,Forest Gate,TQ405855
5,Newham,Little Ilford,TQ435855
6,Newham,Manor Park,TQ425855
7,Newham,Maryland,TQ391849
8,Newham,North Woolwich,TQ435795
9,Newham,Plaistow,TQ405825


#### Create a DataFrame from the list of href links

In [88]:
df_links = pd.DataFrame(href_links_list, columns=['Code','href'])
df_links

,Code,href
0,TQ435815,https://tools.wmflabs.org/geohack/en/51.514205...
1,TQ405815,https://tools.wmflabs.org/geohack/en/51.514959...
2,TQ408807,https://tools.wmflabs.org/geohack/en/51.507695...
3,TQ425835,https://tools.wmflabs.org/geohack/en/51.532429...
4,TQ405855,https://tools.wmflabs.org/geohack/en/51.550902...
5,TQ435855,https://tools.wmflabs.org/geohack/en/51.550147...
6,TQ425855,https://tools.wmflabs.org/geohack/en/51.550401...
7,TQ391849,https://tools.wmflabs.org/geohack/en/51.545857...
8,TQ435795,https://tools.wmflabs.org/geohack/en/51.496234...
9,TQ405825,https://tools.wmflabs.org/geohack/en/51.523944...


#### Merge the Areas and href Links DataFrames

In [89]:
cols = df_links.columns.difference(df_areas.columns)
cols
df_areas_links = pd.concat([df_areas, df_links[cols]], axis=1)
df_areas_links.shape
df_areas_links

,Borough,Area,Code,href
0,Newham,Beckton,TQ435815,https://tools.wmflabs.org/geohack/en/51.514205...
1,Newham,Canning Town,TQ405815,https://tools.wmflabs.org/geohack/en/51.514959...
2,Newham,Custom House,TQ408807,https://tools.wmflabs.org/geohack/en/51.507695...
3,Newham,East Ham,TQ425835,https://tools.wmflabs.org/geohack/en/51.532429...
4,Newham,Forest Gate,TQ405855,https://tools.wmflabs.org/geohack/en/51.550902...
5,Newham,Little Ilford,TQ435855,https://tools.wmflabs.org/geohack/en/51.550147...
6,Newham,Manor Park,TQ425855,https://tools.wmflabs.org/geohack/en/51.550401...
7,Newham,Maryland,TQ391849,https://tools.wmflabs.org/geohack/en/51.545857...
8,Newham,North Woolwich,TQ435795,https://tools.wmflabs.org/geohack/en/51.496234...
9,Newham,Plaistow,TQ405825,https://tools.wmflabs.org/geohack/en/51.523944...


#### Remove the row where there is no data

In [90]:
df_areas_links = df_areas_links.dropna(how='any')
df_areas_links

,Borough,Area,Code,href
0,Newham,Beckton,TQ435815,https://tools.wmflabs.org/geohack/en/51.514205...
1,Newham,Canning Town,TQ405815,https://tools.wmflabs.org/geohack/en/51.514959...
2,Newham,Custom House,TQ408807,https://tools.wmflabs.org/geohack/en/51.507695...
3,Newham,East Ham,TQ425835,https://tools.wmflabs.org/geohack/en/51.532429...
4,Newham,Forest Gate,TQ405855,https://tools.wmflabs.org/geohack/en/51.550902...
5,Newham,Little Ilford,TQ435855,https://tools.wmflabs.org/geohack/en/51.550147...
6,Newham,Manor Park,TQ425855,https://tools.wmflabs.org/geohack/en/51.550401...
7,Newham,Maryland,TQ391849,https://tools.wmflabs.org/geohack/en/51.545857...
8,Newham,North Woolwich,TQ435795,https://tools.wmflabs.org/geohack/en/51.496234...
9,Newham,Plaistow,TQ405825,https://tools.wmflabs.org/geohack/en/51.523944...


Get the geo co-ordinates for all the areas in the Newham borough

In [93]:
geo_codes = []
for row in df_areas_links.itertuples():
    url = row.href
    code = row.Code
    res = requests.get(url).text
    soup1 = BeautifulSoup(res,'lxml')
    
    for lat in soup1.find('span',{'class':'latitude'}):
        latitude = lat
        latitude = float(latitude)
            
    for long in soup1.find('span',{'class':'longitude'}):    
        longitude = long
        longitude = float(longitude)
        
    geo_codes.append((code, latitude, longitude))

print(geo_codes)

[('TQ435815', 51.514206, 0.066634), ('TQ405815', 51.514959, 0.023429), ('TQ408807', 51.507696, 0.027431), ('TQ425835', 51.53243, 0.053041), ('TQ405855', 51.550902, 0.025024), ('TQ435855', 51.550148, 0.068263), ('TQ425855', 51.550401, 0.05385), ('TQ391849', 51.545857, 0.004608), ('TQ435795', 51.496234, 0.065821), ('TQ405825', 51.523945, 0.023828), ('TQ415795', 51.496738, 0.037029), ('TQ385845', 51.54241, -0.004196), ('TQ405837', 51.534728, 0.024306), ('TQ405837', 51.534728, 0.024306)]


In [94]:
df_geo_codes = pd.DataFrame(geo_codes, columns=['Code','Latitude','Longitude'])
df_geo_codes

,Code,Latitude,Longitude
0,TQ435815,51.514206,0.066634
1,TQ405815,51.514959,0.023429
2,TQ408807,51.507696,0.027431
3,TQ425835,51.532430,0.053041
4,TQ405855,51.550902,0.025024
5,TQ435855,51.550148,0.068263
6,TQ425855,51.550401,0.053850
7,TQ391849,51.545857,0.004608
8,TQ435795,51.496234,0.065821
9,TQ405825,51.523945,0.023828


Now merge the Areas(Neighborhoods) and Geocodes DataFrames

In [95]:
df_areas.columns

Index(['Borough', 'Area', 'Code'], dtype='object')

In [96]:
df_areas.shape

(14, 3)

In [97]:
df_geo_codes.columns

Index(['Code', 'Latitude', 'Longitude'], dtype='object')

In [98]:
df_geo_codes.shape

(14, 3)

In [99]:
cols = df_geo_codes.columns.difference(df_areas.columns)

In [100]:
cols

Index(['Latitude', 'Longitude'], dtype='object')

In [101]:
Newham_borough = pd.concat([df_areas, df_geo_codes[cols]], axis=1)

In [102]:
Newham_borough

,Borough,Area,Code,Latitude,Longitude
0,Newham,Beckton,TQ435815,51.514206,0.066634
1,Newham,Canning Town,TQ405815,51.514959,0.023429
2,Newham,Custom House,TQ408807,51.507696,0.027431
3,Newham,East Ham,TQ425835,51.532430,0.053041
4,Newham,Forest Gate,TQ405855,51.550902,0.025024
5,Newham,Little Ilford,TQ435855,51.550148,0.068263
6,Newham,Manor Park,TQ425855,51.550401,0.053850
7,Newham,Maryland,TQ391849,51.545857,0.004608
8,Newham,North Woolwich,TQ435795,51.496234,0.065821
9,Newham,Plaistow,TQ405825,51.523945,0.023828


Change the name of the column 'Area' to 'Neighborhood '

In [103]:
Newham_borough = Newham_borough.rename(columns={'Area' :'Neighborhood'})

In [104]:
Newham_borough.drop(['Code'], axis=1, inplace=True)

In [105]:
Newham_borough.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [106]:
Newham_borough

,Borough,Neighborhood,Latitude,Longitude
0,Newham,Beckton,51.514206,0.066634
1,Newham,Canning Town,51.514959,0.023429
2,Newham,Custom House,51.507696,0.027431
3,Newham,East Ham,51.532430,0.053041
4,Newham,Forest Gate,51.550902,0.025024
5,Newham,Little Ilford,51.550148,0.068263
6,Newham,Manor Park,51.550401,0.053850
7,Newham,Maryland,51.545857,0.004608
8,Newham,North Woolwich,51.496234,0.065821
9,Newham,Plaistow,51.523945,0.023828


Now lets get the Coordinates of Newham Borough

In [107]:
address = 'Newham, London'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Newham are {}, {}.'.format(latitude, longitude))

C:\Users\AID-FLOATER-2015-02\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Newham are 51.52999955, 0.0293179602938221.


Let's Visualize the Areas(Neighborhoods) of Newham Borough

In [108]:
# create map of Newham using latitude and longitude values
map_Newham = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Newham_borough['Latitude'], Newham_borough['Longitude'], Newham_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Newham)  
    
map_Newham

Define FourSquare Credentials
Let's explore the first Neighborhood(Area) in Newham Borough

In [109]:
Newham_borough.shape

(14, 4)

In [110]:
Newham_borough.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [111]:
Newham_borough.loc[0, 'Neighborhood']

'Beckton'

In [112]:
Newham_borough.loc[0]

Borough           Newham
Neighborhood     Beckton
Latitude         51.5142
Longitude       0.066634
Name: 0, dtype: object

In [113]:
neighborhood_latitude = Newham_borough.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Newham_borough.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Newham_borough.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beckton are 51.514206, 0.066634.


Now get the top 100 places of Newham Neighborhood within 500KM radius

In [114]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret=\
       {}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [115]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5de7304483525f001b8ce1b5'},
 'response': {'headerLocation': 'Beckton',
  'headerFullLocation': 'Beckton, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 51.518706004500004,
    'lng': 0.07385150597103936},
   'sw': {'lat': 51.5097059955, 'lng': 0.05941649402896063}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e0c524018a89ad010688d2b',
       'name': 'East london Gymnastics Club',
       'location': {'lat': 51.514106774737556,
        'lng': 0.060155068624099396,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.514106774737556,
          'lng': 0.060155068624099396}],
        'distance': 448,
        'cc': 'GB',
        'city': 'East Ham',
       

Extract the Categories of the Venues

In [116]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [117]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,East london Gymnastics Club,Gym / Fitness Center,51.514107,0.060155
1,Home Bargains,Discount Store,51.516805,0.062804
2,Lituanica,Grocery Store,51.516442,0.062927
3,Premier Inn London Beckton,Hotel,51.515125,0.061209
4,Matalan,Clothing Store,51.516004,0.062635


In [118]:

nearby_venues.shape

(9, 4)

In [119]:
nearby_venues['name'].value_counts()

East london Gymnastics Club    1
Dreams                         1
Lituanica                      1
Premier Inn London Beckton     1
Matalan                        1
Beckton DLR Station            1
Brewers Fayre                  1
Beckton Retail Park            1
Home Bargains                  1
Name: name, dtype: int64

In [120]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


#### Explore Neighborhoods in Newham Borough

In [121]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [122]:
Newham_venues = getNearbyVenues(names=Newham_borough['Neighborhood'],
                                   latitudes=Newham_borough['Latitude'],
                                   longitudes=Newham_borough['Longitude']
                                  )

Beckton
Canning Town
Custom House
East Ham
Forest Gate
Little Ilford
Manor Park
Maryland
North Woolwich
Plaistow
Silvertown
Stratford
Upton Park
West Ham


In [123]:
Newham_venues.shape

(221, 7)

In [124]:
Newham_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beckton,51.514206,0.066634,East london Gymnastics Club,51.514107,0.060155,Gym / Fitness Center
1,Beckton,51.514206,0.066634,Home Bargains,51.516805,0.062804,Discount Store
2,Beckton,51.514206,0.066634,Lituanica,51.516442,0.062927,Grocery Store
3,Beckton,51.514206,0.066634,Premier Inn London Beckton,51.515125,0.061209,Hotel
4,Beckton,51.514206,0.066634,Matalan,51.516004,0.062635,Clothing Store



Let's check how many venues were returned for each neighborhood

In [125]:
Newham_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beckton,9,9,9,9,9,9
Canning Town,5,5,5,5,5,5
Custom House,28,28,28,28,28,28
East Ham,13,13,13,13,13,13
Forest Gate,11,11,11,11,11,11
Little Ilford,5,5,5,5,5,5
Manor Park,4,4,4,4,4,4
Maryland,24,24,24,24,24,24
North Woolwich,23,23,23,23,23,23


In [126]:
print('There are {} uniques categories.'.format(len(Newham_venues['Venue Category'].unique())))

There are 99 uniques categories.


#### Analyze Each Neighborhood in Newham borough

In [127]:
# one hot encoding
Newham_onehot = pd.get_dummies(Newham_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Newham_onehot['Neighborhood'] = Newham_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Newham_onehot.columns[-1]] + list(Newham_onehot.columns[:-1])
Newham_onehot = Newham_onehot[fixed_columns]

Newham_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fountain,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,German Restaurant,Go Kart Track,Grocery Store,Gym / Fitness Center,Gym Pool,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Moving Target,Multiplex,Museum,Paintball Field,Park,Pharmacy,Pier,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Shopping Plaza,Steakhouse,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [128]:
Newham_onehot.shape

(221, 100)

In [129]:
Newham_grouped = Newham_onehot.groupby('Neighborhood').mean().reset_index()
Newham_grouped

,Neighborhood,Accessories Store,African Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Boutique,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fountain,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,German Restaurant,Go Kart Track,Grocery Store,Gym / Fitness Center,Gym Pool,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Moving Target,Multiplex,Museum,Paintball Field,Park,Pharmacy,Pier,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Scenic Lookout,Sculpture Garden,Shoe Store,Shopping Mall,Shopping Plaza,Steakhouse,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Beckton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.111111,0.00,0.000000,0.000000,0.000,0.111111,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Canning Town,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.200,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.20,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Custom House,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.035714,0.000000,0.071429,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.035714,0.000000,0.178571,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.035714,0.035714,0.000000,0.071429,0.035714,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.035714,0.000000,0.000,0.000000,0.00000

In [130]:
Newham_grouped.columns

Index(['Neighborhood', 'Accessories Store', 'African Restaurant',
       'Art Gallery', 'Asian Restaurant', 'Bakery', 'Bar', 'Boat or Ferry',
       'Bookstore', 'Boutique', 'Brewery', 'Bridge', 'Bubble Tea Shop',
       'Burger Joint', 'Bus Line', 'Bus Stop', 'Business Service', 'Café',
       'Chinese Restaurant', 'Clothing Store', 'Coffee Shop',
       'Construction & Landscaping', 'Convenience Store', 'Cosmetics Shop',
       'Creperie', 'Dance Studio', 'Department Store', 'Dessert Shop', 'Diner',
       'Discount Store', 'Doner Restaurant', 'Eastern European Restaurant',
       'Electronics Store', 'English Restaurant', 'Fast Food Restaurant',
       'Fish & Chips Shop', 'Fountain', 'Fried Chicken Joint',
       'Furniture / Home Store', 'Gas Station', 'General Entertainment',
       'German Restaurant', 'Go Kart Track', 'Grocery Store',
       'Gym / Fitness Center', 'Gym Pool', 'Hotel', 'Hotel Bar',
       'Ice Cream Shop', 'Indian Restaurant', 'Indie Movie Theater',
       'Ind

#### Let's print each neighborhood along with the top 5 most common venues

In [131]:
num_top_venues = 5
for hood in Newham_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Newham_grouped[Newham_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Beckton----
                    venue  freq
0  Furniture / Home Store  0.11
1      Light Rail Station  0.11
2                     Pub  0.11
3          Shopping Plaza  0.11
4          Clothing Store  0.11


----Canning Town----
                        venue  freq
0                 Gas Station   0.2
1           Convenience Store   0.2
2  Construction & Landscaping   0.2
3                        Park   0.2
4            Business Service   0.2


----Custom House----
            venue  freq
0           Hotel  0.18
1            Café  0.07
2             Pub  0.07
3  Sandwich Place  0.07
4     Coffee Shop  0.07


----East Ham----
               venue  freq
0     Clothing Store  0.15
1  Electronics Store  0.08
2           Gym Pool  0.08
3        Coffee Shop  0.08
4               Park  0.08


----Forest Gate----
                  venue  freq
0         Grocery Store  0.27
1                   Pub  0.09
2                  Café  0.09
3                Bakery  0.09
4  Fast Food Restaurant  0.09


-

In [132]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [134]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Newham_grouped['Neighborhood']

for ind in np.arange(Newham_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Newham_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Hotel,Gym / Fitness Center,Grocery Store,Pub,Light Rail Station,Discount Store,Furniture / Home Store,Clothing Store,Shopping Plaza,Wine Bar
1,Canning Town,Park,Convenience Store,Construction & Landscaping,Gas Station,Business Service,Wine Bar,Fish & Chips Shop,Discount Store,Doner Restaurant,Eastern European Restaurant
2,Custom House,Hotel,Coffee Shop,Café,Pub,Sandwich Place,Scenic Lookout,Hotel Bar,Italian Restaurant,Japanese Restaurant,Light Rail Station
3,East Ham,Clothing Store,Fast Food Restaurant,Electronics Store,Warehouse Store,Gym Pool,Park,Grocery Store,Discount Store,Pub,Coffee Shop
4,Forest Gate,Grocery Store,Market,Fish & Chips Shop,Bakery,Pub,Train Station,Moving Target,Café,Fast Food Restaurant,Diner
5,Little Ilford,Ice Cream Shop,Asian Restaurant,Fried Chicken Joint,Grocery Store,Indian Restaurant,Gym Pool,Gym / Fitness Center,Dessert Shop,Diner,Discount Store
6,Manor Park,Gym / Fitness Center,Turkish Restaurant,Restaurant,Gas Station,Wine Bar,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Discount Store
7,Maryland,Hotel,Pub,Grocery Store,Café,Sculpture Garden,Indian Restaurant,Indie Theater,General Entertainment,Furniture / Home Store,Liquor Store
8,North Woolwich,Pier,Hotel,Clothing Store,Shoe Store,Gym / Fitness Center,Warehouse Store,Italian Restaurant,Fountain,Pharmacy,Plaza
9,Plaistow,Park,Grocery Store,Pub,Café,Business Service,Bus Stop,Indian Restaurant,Gym / Fitness Center,African Restaurant,Vietnamese Restaurant


#### Cluster the Neighborhoods

In [135]:
# set number of clusters
kclusters = 5

Newham_grouped_clustering = Newham_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Newham_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 3, 2, 0, 0, 0])

In [136]:
Newham_merged = Newham_borough

# add clustering labels
Newham_merged['Cluster Labels'] = kmeans.labels_

# merge Neighborhoods dataframe with Newham borough dataframe to add latitude/longitude for each neighborhood
Newham_merged = Newham_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Newham_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Newham,Beckton,51.514206,0.066634,0,Hotel,Gym / Fitness Center,Grocery Store,Pub,Light Rail Station,Discount Store,Furniture / Home Store,Clothing Store,Shopping Plaza,Wine Bar
1,Newham,Canning Town,51.514959,0.023429,4,Park,Convenience Store,Construction & Landscaping,Gas Station,Business Service,Wine Bar,Fish & Chips Shop,Discount Store,Doner Restaurant,Eastern European Restaurant
2,Newham,Custom House,51.507696,0.027431,0,Hotel,Coffee Shop,Café,Pub,Sandwich Place,Scenic Lookout,Hotel Bar,Italian Restaurant,Japanese Restaurant,Light Rail Station
3,Newham,East Ham,51.532430,0.053041,0,Clothing Store,Fast Food Restaurant,Electronics Store,Warehouse Store,Gym Pool,Park,Grocery Store,Discount Store,Pub,Coffee Shop
4,Newham,Forest Gate,51.550902,0.025024,0,Grocery Store,Market,Fish & Chips Shop,Bakery,Pub,Train Station,Moving Target,Café,Fast Food Restaurant,Diner


Let's Visualize the Cluster

In [137]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Newham_merged['Latitude'], Newham_merged['Longitude'], Newham_merged['Neighborhood'], Newham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the Clusters

In [138]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 0, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Hotel,Gym / Fitness Center,Grocery Store,Pub,Light Rail Station,Discount Store,Furniture / Home Store,Clothing Store,Shopping Plaza,Wine Bar
2,Custom House,Hotel,Coffee Shop,Café,Pub,Sandwich Place,Scenic Lookout,Hotel Bar,Italian Restaurant,Japanese Restaurant,Light Rail Station
3,East Ham,Clothing Store,Fast Food Restaurant,Electronics Store,Warehouse Store,Gym Pool,Park,Grocery Store,Discount Store,Pub,Coffee Shop
4,Forest Gate,Grocery Store,Market,Fish & Chips Shop,Bakery,Pub,Train Station,Moving Target,Café,Fast Food Restaurant,Diner
7,Maryland,Hotel,Pub,Grocery Store,Café,Sculpture Garden,Indian Restaurant,Indie Theater,General Entertainment,Furniture / Home Store,Liquor Store
8,North Woolwich,Pier,Hotel,Clothing Store,Shoe Store,Gym / Fitness Center,Warehouse Store,Italian Restaurant,Fountain,Pharmacy,Plaza
9,Plaistow,Park,Grocery Store,Pub,Café,Business Service,Bus Stop,Indian Restaurant,Gym / Fitness Center,African Restaurant,Vietnamese Restaurant
11,Stratford,Pub,Sandwich Place,Clothing Store,Burger Joint,General Entertainment,Pizza Place,Department Store,Shopping Mall,Cosmetics Shop,Café


In [139]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 1, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upton Park,Pub,Bus Line,Convenience Store,Boutique,Bus Stop,Wine Bar,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store
13,West Ham,Pub,Bus Line,Convenience Store,Boutique,Bus Stop,Wine Bar,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store


In [140]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 2, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Manor Park,Gym / Fitness Center,Turkish Restaurant,Restaurant,Gas Station,Wine Bar,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Discount Store


In [141]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 3, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Little Ilford,Ice Cream Shop,Asian Restaurant,Fried Chicken Joint,Grocery Store,Indian Restaurant,Gym Pool,Gym / Fitness Center,Dessert Shop,Diner,Discount Store


In [142]:
Newham_merged.loc[Newham_merged['Cluster Labels'] == 4, Newham_merged.columns[[1] + list(range(5, Newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Canning Town,Park,Convenience Store,Construction & Landscaping,Gas Station,Business Service,Wine Bar,Fish & Chips Shop,Discount Store,Doner Restaurant,Eastern European Restaurant
10,Silvertown,Gym / Fitness Center,Park,Go Kart Track,Paintball Field,Construction & Landscaping,Museum,Theater,Café,Wine Bar,English Restaurant


### Conclusion
#### now that we have examined the 5 clusters, it can be recommended that Beckton, Costum House, Maryland, Eastham and Manor Park are best options to open a Persiaon restuarant (as they have the most asian population). also in these areas, hotels are used highly and this there would be higher tourist attention to the retuarants.